In [1]:
from datasets import load_dataset, Dataset, load_metric
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, AutoConfig, LlamaTokenizer, MistralConfig
#from transformers import AdapterConfig, AdapterType
from transformers import TrainingArguments, Trainer, BitsAndBytesConfig, DataCollatorWithPadding
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import csv
import re
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from nltk.translate.bleu_score import corpus_bleu
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, hamming_loss, accuracy_score
from peft import PeftModel, PeftConfig
import sacrebleu
import logging
import bitsandbytes as bnb
from peft import PeftModel
from bitsandbytes.optim import GlobalOptimManager
#from trl import SFTTrainer
import optimum
#import adapter-transformers
from torch.nn import DataParallel
logging.basicConfig(level=logging.INFO)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
df = pd.read_csv("translated_final1.csv") 
# Process the dataframe to fit the gpteacher format
df['input'] = df[['flags', 'intent', 'category']].agg(', '.join, axis=1)
df1 = df[['instruction', 'input', 'response']]
df1.to_parquet('bitext.parquet', index=False)
df2 = df1.sample(frac=0.1, random_state=42)
train, temp = train_test_split(df2, test_size=0.3, random_state=42)
val, test = train_test_split(temp, test_size=0.5, random_state=42)
train.to_parquet('train.parquet', index=False)
val.to_parquet('val.parquet', index=False)
test.to_parquet('test.parquet', index=False)

In [ ]:
# Creating a Mistral model with custom configuration
#config = MistralConfig()
#model_id = "Ichsan2895/Merak-7B-v4"
#model = AutoModelForCausalLM.from_config(config)

# Load the pretrained weights into the model
# Note: This step assumes that the custom configuration is compatible with the Merak model weights
#model.load_state_dict(AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True, torch_dtype=torch.bfloat16, device_map="auto", pad_token_id=0, attn_implementation="flash_attention_2", cache_dir="./model_cache").state_dict())


# Saving the model
#model_save_path = "/workspace/axolotl/merak_model"
#model.save_pretrained(model_save_path)

In [4]:
qlora_model = './axolotl/qlora-out'
model_id = "Ichsan2895/Merak-7B-v4"
#config = MistralConfig() 
#model1 = AutoModelForCausalLM.from_config(config)
model1 = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="auto", pad_token_id=0, attn_implementation="flash_attention_2", cache_dir="/workspace")
model1 = PeftModel.from_pretrained(model1, qlora_model)
model1 = model1.merge_and_unload()

# Saving the model
model_save_path1 = "/workspace/axolotl/merged_model"
model1.save_pretrained(model_save_path1)

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [5]:
model_save_path1 = "/workspace/axolotl/merged_model"
model1 = AutoModelForCausalLM.from_pretrained(model_save_path1, torch_dtype=torch.bfloat16, device_map="auto", pad_token_id=0, attn_implementation="flash_attention_2")
model_id = "Ichsan2895/Merak-7B-v4"
tokenizer = LlamaTokenizer.from_pretrained(model_id)

def generate_response(question: str) -> str:
    chat = [
      {"role": "system", "content": "Ada yang bisa saya bantu?"},
      {"role": "user", "content": question},
    ]

    prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

    inputs = tokenizer(prompt, return_tensors="pt", return_attention_mask=True)

    with torch.no_grad():
        outputs = model1.generate(input_ids=inputs["input_ids"].to("cuda"),
                           attention_mask=inputs.attention_mask,
                           eos_token_id=tokenizer.eos_token_id,
                           pad_token_id=tokenizer.eos_token_id,
                           max_new_tokens=256)
        response = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]

        assistant_start = f'''{question} \n assistant\n '''
        response_start = response.find(assistant_start)
        return response[response_start + len(assistant_start) :].strip()

prompt = "bagaimana saya dapat membatalkan pembelian saya?"
print(generate_response(prompt))

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1518: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


Saya mengerti bahwa Anda mencari bantuan dengan membatalkan pembelian Anda. Untuk membatalkan pembelian Anda, Anda dapat menghubungi tim pendukung pelanggan kami melalui situs web kami atau dengan menghubungi saluran dukungan pelanggan kami. Mereka akan dapat membimbing Anda melalui proses dan memastikan pembatalan yang halus. Jika Anda memiliki pertanyaan atau kekhawatiran lebih lanjut, jangan ragu untuk bertanya.


In [8]:
model_id = "Ichsan2895/Merak-7B-v4"
config = AutoConfig.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             device_map="auto",
                                             trust_remote_code=True, cache_dir="/workspace")

tokenizer = LlamaTokenizer.from_pretrained(model_id)

def generate_response(question: str) -> str:
    chat = [
      {"role": "system", "content": "Anda adalah Merak, sebuah model kecerdasan buatan yang dilatih oleh Muhammad Ichsan. Mohon jawab pertanyaan berikut dengan benar, faktual, dan ramah."},
      {"role": "user", "content": question},
    ]

    prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

    inputs = tokenizer(prompt, return_tensors="pt", return_attention_mask=True)

    with torch.no_grad():
        outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"),
                           attention_mask=inputs.attention_mask,
                           eos_token_id=tokenizer.eos_token_id,
                           pad_token_id=tokenizer.eos_token_id,
                           max_new_tokens=256)
        response = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]

        assistant_start = f'''{question} \n assistant\n '''
        response_start = response.find(assistant_start)
        return response[response_start + len(assistant_start) :].strip()

prompt = "bagaimana saya dapat membatalkan pembelian saya?"
print(generate_response(prompt))

config.json:   0%|          | 0.00/600 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Untuk membatalkan pembelian Anda, Anda dapat mengikuti langkah-langkah ini:

1. Buka aplikasi atau situs web Amazon.

2. Klik pada ikon pengaturan di sudut kanan atas.

3. Klik pada "Pesan dan Pengiriman" di bagian bawah halaman.

4. Klik pada pesanan yang ingin Anda batalkan.

5. Klik pada tombol "Batalkan Pesanan" di bagian atas halaman.

6. Klik pada "Batalkan Pesanan" di dialog konfirmasi.

7. Pilih alasan untuk membatalkan pesanan Anda.

8. Klik pada "Batalkan Pesanan" untuk mengonfirmasi.

Perhatikan bahwa jika Anda membatalkan pesanan Anda, Anda mungkin mungkin dikenakan


In [6]:
model_save_path1 = "/workspace/axolotl/merged_model"
model1 = AutoModelForCausalLM.from_pretrained(model_save_path1, torch_dtype=torch.bfloat16, device_map="auto", pad_token_id=0, attn_implementation="flash_attention_2")
model_id = "Ichsan2895/Merak-7B-v4"
tokenizer = LlamaTokenizer.from_pretrained(model_id) 

# Function to generate responses for a list of questions
def generate_responses(questions, model, tokenizer):
    model.eval()
    responses = []
    for question in questions:
        chat = [
            {"role": "system", "content": "Ada yang bisa saya bantu?"},
            {"role": "user", "content": question},
        ]
        prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
        inputs = tokenizer(prompt, return_tensors="pt", return_attention_mask=True)
        inputs = {k: v.to('cuda') for k, v in inputs.items()}
        with torch.no_grad():
            outputs = model.generate(**inputs, max_new_tokens=128, num_return_sequences=1)
            response = tokenizer.decode(outputs[0], skip_special_tokens=True)
            assistant_start = f'''{question} \n assistant\n '''
            response_start = response.find(assistant_start)
            responses.append(response[response_start + len(assistant_start) :].strip())
    return responses

# Function to compute evaluation metrics
def compute_metrics(df, model, tokenizer):
    questions = df['instruction'].tolist()
    generated_responses = generate_responses(questions, model, tokenizer)
    references = [ref.split() for ref in df['response'].tolist()]
    hypotheses = [resp.split() for resp in generated_responses]

    bleu_score = corpus_bleu([[ref] for ref in references], hypotheses)
    chrf_metric = sacrebleu.metrics.CHRF(word_order=0)
    chrfpp_metric = sacrebleu.metrics.CHRF(word_order=2)

    chrf_score = chrf_metric.corpus_score(generated_responses, [[ref] for ref in df['response'].tolist()])
    chrfpp_score = chrfpp_metric.corpus_score(generated_responses, [[ref] for ref in df['response'].tolist()])

    return bleu_score, chrf_score.score, chrfpp_score.score

# Load the validation and test datasets
val_df = pd.read_parquet('val.parquet')
test_df = pd.read_parquet('test.parquet')

# Compute metrics for validation and test datasets
val_bleu, val_chrf, val_chrfpp = compute_metrics(val_df, model1, tokenizer)
test_bleu, test_chrf, test_chrfpp = compute_metrics(test_df, model1, tokenizer)

print("Validation Set Metrics:")
print("BLEU Score:", val_bleu)
print("chrF Score:", val_chrf)
print("chrF++ Score:", val_chrfpp)

print("\nTest Set Metrics:")
print("BLEU Score:", test_bleu)
print("chrF Score:", test_chrf)
print("chrF++ Score:", test_chrfpp)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1518: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


Validation Set Metrics:
BLEU Score: 0.06850437696225994
chrF Score: 49.27643486537506
chrF++ Score: 45.75078773862919

Test Set Metrics:
BLEU Score: 0.062102389753086004
chrF Score: 51.626813741243595
chrF++ Score: 49.04707207666462


In [12]:
#BNB_CONFIG = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.bfloat16, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4")
model_save_path1 = "/workspace/axolotl/merged_model"
model1 = AutoModelForCausalLM.from_pretrained(model_save_path1, torch_dtype=torch.bfloat16, device_map="auto", pad_token_id=0, attn_implementation="flash_attention_2"
                                             #, quantization_config=BNB_CONFIG 
                                             )
model_id = "Ichsan2895/Merak-7B-v4"
tokenizer = LlamaTokenizer.from_pretrained(model_id) 

def generate_responses_in_batches(questions, model, tokenizer, batch_size=32):
    print(f"Using batch size: {batch_size}")
    model.eval()
    responses = []

    for i in range(0, len(questions), batch_size):
        batch_questions = questions[i:i + batch_size]

        # Generate prompts for each question in the batch
        batch_prompts = []
        for question in batch_questions:
            chat = [{"role": "system", "content": "Ada yang bisa saya bantu?"}, {"role": "user", "content": question}]
            prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
            batch_prompts.append(prompt)

        # Tokenize each prompt and collate them into a batch
        batch_inputs = tokenizer(batch_prompts, return_tensors="pt", padding=True, truncation=True, max_length=512)
        batch_inputs = {k: v.to('cuda') for k, v in batch_inputs.items()}

        with torch.no_grad():
            batch_outputs = model.generate(**batch_inputs, max_new_tokens=128, num_return_sequences=1)
            batch_responses = tokenizer.batch_decode(batch_outputs, skip_special_tokens=True)

            for response in batch_responses:
                assistant_start = "assistant\n"  # Simplified start pattern
                response_start = response.find(assistant_start)
                responses.append(response[response_start + len(assistant_start):].strip())

    return responses

def compute_metrics1(df, model, tokenizer, batch_size=32):
    questions = df['instruction'].tolist()
    generated_responses = generate_responses_in_batches(questions, model, tokenizer, batch_size)
    references = [ref.split() for ref in df['response'].tolist()]
    hypotheses = [resp.split() for resp in generated_responses]

    bleu_score = corpus_bleu([[ref] for ref in references], hypotheses)
    chrf_metric = sacrebleu.metrics.CHRF(word_order=0)
    chrfpp_metric = sacrebleu.metrics.CHRF(word_order=2)

    chrf_score = chrf_metric.corpus_score(generated_responses, [[ref] for ref in df['response'].tolist()])
    chrfpp_score = chrfpp_metric.corpus_score(generated_responses, [[ref] for ref in df['response'].tolist()])

    return bleu_score, chrf_score.score, chrfpp_score.score

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [13]:
# Load the validation and test datasets
val_df = pd.read_parquet('val.parquet')
test_df = pd.read_parquet('test.parquet')

# Define batch size
batch_size = 32  # Adjust based on your GPU memory

# Compute metrics for validation and test datasets
val_bleu, val_chrf, val_chrfpp = compute_metrics1(val_df, model1, tokenizer, batch_size)
test_bleu, test_chrf, test_chrfpp = compute_metrics1(test_df, model1, tokenizer, batch_size)

print("Validation Set Metrics:")
print("BLEU Score:", val_bleu)
print("chrF Score:", val_chrf)
print("chrF++ Score:", val_chrfpp)

print("\nTest Set Metrics:")
print("BLEU Score:", test_bleu)
print("chrF Score:", test_chrf)
print("chrF++ Score:", test_chrfpp)

Using batch size: 32


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1518: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


Using batch size: 32
Validation Set Metrics:
BLEU Score: 0.03390882789020255
chrF Score: 49.27643486537506
chrF++ Score: 45.75078773862919

Test Set Metrics:
BLEU Score: 0.034053665983892346
chrF Score: 45.36736245520675
chrF++ Score: 41.8263441561195


In [ ]:
model_save_path1 = "/workspace/axolotl/merged_model"
model1 = AutoModelForCausalLM.from_pretrained(model_save_path1, torch_dtype=torch.bfloat16, device_map="auto", pad_token_id=0, attn_implementation="flash_attention_2")
model_id = "Ichsan2895/Merak-7B-v4"
tokenizer = LlamaTokenizer.from_pretrained(model_id) 

def generate_responses_in_batches(questions, model, tokenizer, batch_size=32):
    print(f"Using batch size: {batch_size}")
    model.eval()
    responses = []

    for i in range(0, len(questions), batch_size):
        batch_questions = questions[i:i + batch_size]

        # Generate prompts and responses for each question in the batch
        batch_responses = []
        for question in batch_questions:
            chat = [
                {"role": "system", "content": "Ada yang bisa saya bantu?"},
                {"role": "user", "content": question},
            ]
            prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
            inputs = tokenizer(prompt, return_tensors="pt", return_attention_mask=True)
            inputs = {k: v.to('cuda') for k, v in inputs.items()}
            with torch.no_grad():
                output = model.generate(**inputs, max_new_tokens=128, num_return_sequences=1)
                response = tokenizer.decode(output[0], skip_special_tokens=True)
                responses.append(response)

    return responses

def compute_metrics1(df, model, tokenizer, batch_size=32):
    questions = df['instruction'].tolist()
    generated_responses = generate_responses_in_batches(questions, model, tokenizer, batch_size)
    references = [ref.split() for ref in df['response'].tolist()]
    hypotheses = [resp.split() for resp in generated_responses]

    bleu_score = corpus_bleu([[ref] for ref in references], hypotheses)
    chrf_metric = sacrebleu.metrics.CHRF(word_order=0)
    chrfpp_metric = sacrebleu.metrics.CHRF(word_order=2)

    chrf_score = chrf_metric.corpus_score(generated_responses, [[ref] for ref in df['response'].tolist()])
    chrfpp_score = chrfpp_metric.corpus_score(generated_responses, [[ref] for ref in df['response'].tolist()])

    return bleu_score, chrf_score.score, chrfpp_score.score

# Load the validation and test datasets
val_df = pd.read_parquet('val.parquet')
test_df = pd.read_parquet('test.parquet')

# Define batch size
batch_size = 32  # Adjust based on your GPU memory

# Compute metrics for validation and test datasets
val_bleu, val_chrf, val_chrfpp = compute_metrics1(val_df, model1, tokenizer, batch_size)
test_bleu, test_chrf, test_chrfpp = compute_metrics1(test_df, model1, tokenizer, batch_size)

print("Validation Set Metrics:")
print("BLEU Score:", val_bleu)
print("chrF Score:", val_chrf)
print("chrF++ Score:", val_chrfpp)

print("\nTest Set Metrics:")
print("BLEU Score:", test_bleu)
print("chrF Score:", test_chrf)
print("chrF++ Score:", test_chrfpp)

In [ ]:
# Prepare data for BLEU and chrF evaluation
test_questions = test["instruction"].tolist()
generated_responses = generate_responses(test_questions, model1, tokenizer, device)
references = [ref.split() for ref in test["response"].tolist()]
hypotheses = [resp.split() for resp in generated_responses]

# Calculate BLEU score
bleu_score = corpus_bleu([[ref] for ref in references], hypotheses)
print("BLEU Score:", bleu_score)

# Calculate chrF score
chrf = sacrebleu.metrics.CHRF(word_order=0)
chrf_score = chrf.corpus_score(generated_responses, [[ref] for ref in test["response"].tolist()])
print("chrF Score:", chrf_score.score)

# Calculate chrF++ score
chrfpp = sacrebleu.metrics.CHRF(word_order=2)
chrfpp_score = chrfpp.corpus_score(generated_responses, [[ref] for ref in test["response"].tolist()])
print("chrF++ Score:", chrfpp_score.score) 